In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from torcheval.metrics import R2Score
from torchmetrics.functional import r2_score
import pandas as pd
import torch
from torch.utils.data import random_split
import shutil
import os
from dataset import PGLSDataset
from models import PGLSModel, EnsemblePGLSModel
import timm

/home/piotr/.local/lib/python3.10/site-packages/fbgemm_gpu/fbgemm_gpu_py.so: undefined symbol: _ZNK3c105Error4whatEv
/home/piotr/.local/lib/python3.10/site-packages/fbgemm_gpu/experimental/gen_ai/fbgemm_gpu_experimental_gen_ai_py.so: cannot open shared object file: No such file or directory


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
torch.cuda.empty_cache()

In [4]:
for folder in ["data/train_images", "data/test_images"]:
    if "0" not in os.listdir(folder):
        print("Moving images to 0 folder")
        os.makedirs(f"{folder}/0")
        for filename in os.listdir(folder):
            if filename.lower().endswith(".jpeg"):
                source_path = os.path.join(folder, filename)
                target_path = os.path.join(f"{folder}/0", filename)

                shutil.move(source_path, target_path)

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),         # Convert PIL image to tensor (H x W x C) in the range [0.0, 1.0]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

batch_size = 16     # use 4 if problem with GPU memory

In [6]:
train_images_path = 'data/train_images'
test_images_path = 'data/test_images'

train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'


tabular_data = pd.read_csv(train_csv_path)
targets = ["X4", "X11", "X18", "X26", "X50", "X3112"]

# Filter data
upper_values = {}
for target in targets:
    upper_values[target] = tabular_data[target+"_mean"].quantile(0.99)
    tabular_data = tabular_data[tabular_data[target+"_mean"] < upper_values[target]]
    tabular_data = tabular_data[tabular_data[target+"_mean"] > 0]

# Normalize the targets
original_means = tabular_data[[f"{target}_mean" for target in targets]].mean()
original_stds = tabular_data[[f"{target}_mean" for target in targets]].std()
tabular_data[[f"{target}_mean" for target in targets]] = (tabular_data[[f"{target}_mean" for target in targets]] - original_means) / original_stds

# Normalize the features
tabular_input_size = 0
for column in tabular_data.columns:
    if column in ["id"]+[target+"_mean" for target in targets]+[target+"_sd" for target in targets]:
        continue
    tabular_input_size += 1
    min_val = tabular_data[column].min()
    max_val = tabular_data[column].max()
    tabular_data[column] = (tabular_data[column] - min_val) / (max_val - min_val)


test_tabular_data = pd.read_csv(test_csv_path)
# Normalize the features
for column in test_tabular_data.columns:
    if column in ["id"]:
        continue
    min_val = test_tabular_data[column].min()
    max_val = test_tabular_data[column].max()
    test_tabular_data[column] = (test_tabular_data[column] - min_val) / (max_val - min_val)

train_images_dataset = ImageFolder(root=train_images_path, transform=transform)
test_images_dataset = ImageFolder(root=test_images_path, transform=transform)

train_image_csv_dataset = PGLSDataset(tabular_data=tabular_data, image_folder=train_images_dataset, transform_csv=None)
train, val = random_split(train_image_csv_dataset, [int(0.8*len(train_image_csv_dataset)), len(train_image_csv_dataset) - int(0.8*len(train_image_csv_dataset))])
test_image_csv_dataset = PGLSDataset(tabular_data=test_tabular_data, image_folder=test_images_dataset, transform_csv=None)


train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_image_csv_dataset, batch_size=batch_size, shuffle=False)

In [7]:
original_stds = torch.from_numpy(original_stds.values).float()
original_means = torch.from_numpy(original_means.values).float()
def denormalize_targets(targets):
    return targets * original_stds + original_means

In [11]:
# effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
effnet = timm.create_model(
    'efficientnet_b0.ra_in1k',
    pretrained=True,
    num_classes=0,
)

model = PGLSModel(effnet, tabular_input_size)
metric = R2Score()
criterion = torch.nn.MSELoss()

torch.cuda.empty_cache()


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [12]:
model.load_state_dict(torch.load("models/effnet_best_epoch.pt", map_location=torch.device('cpu')))
model.to(device)

PGLSModel(
  (image_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=

In [13]:
predictions = []
model.eval()
model.to(device)
with torch.no_grad():
    for data in test_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        outputs = outputs.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        predictions.append(outputs_denorm)
predictions = [item for sublist in predictions for item in sublist]

In [ ]:
def save_predictions(predictions, test_csv_dataframe, filename):
    with open(filename, "w") as f:
        f.write("id,X4,X11,X18,X26,X50,X3112\n")
        for pred, id in zip(predictions, test_csv_dataframe["id"]):
            pred = [p.item() for p in pred]
            f.write(f"{id},{','.join([str(p) for p in pred])}\n")

In [ ]:
save_predictions(predictions, test_tabular_data, "predictions.csv")